In [ ]:
import sys
import os
from google.colab import drive
drive.mount('/content/drive')
project_path = '/content/drive/My Drive/CARMS_MF'
os.chdir(project_path)
sys.path.append(project_path)

Mounted at /content/drive


In [ ]:
import warnings
warnings.simplefilter("ignore", category=FutureWarning)

In [ ]:
import pandas as pd
import numpy as np
import duckdb
import itertools
import torch

In [ ]:
from split_temporal_cv import create_temporal_folds_with_cold_start

In [ ]:
from model_standard_mf import Standard_MF
from model_standard_mf_bias import Standard_MF_Bias

from model_carms_mf import CARMS_MF
from model_carms_mf_bias import CARMS_MF_Bias

In [ ]:
from signal_carms_generator_mlxtend import SignalGenerator
# from signal_carms_generator_spark import SignalGenerator

In [ ]:
from eval_ndcg_k import calculate_ndcg_at_k

In [ ]:
torch.cuda.is_available()

True

# Import Data

In [ ]:
df_ml_small = pd.read_csv('./database/ml-latest-small/ratings.csv')
df_ml_small = df_ml_small.rename(columns={'userId':'userid',
                                          'movieId':'itemid',
                                          'rating':'rating'})

df_ml_100k = pd.read_csv('./database/ml-100k/ratings.csv')
df_ml_100k = df_ml_100k.rename(columns={'userId':'userid',
                                        'movieId':'itemid',
                                        'rating':'rating'})

df_ml_1m = pd.read_csv('./database/ml-1m/ratings.csv')
df_ml_1m = df_ml_1m.rename(columns={'userId':'userid',
                                    'movieId':'itemid',
                                    'rating':'rating'})

In [ ]:
train_list_ml_small, test_list_ml_small, cold_start_list_ml_small = \
    create_temporal_folds_with_cold_start(df_ml_small,5)

train_list_ml_100k, test_list_ml_100k, cold_start_list_ml_100k = \
    create_temporal_folds_with_cold_start(df_ml_100k,5)

train_list_ml_1m, test_list_ml_1m, cold_start_list_ml_1m = \
    create_temporal_folds_with_cold_start(df_ml_1m,5)

In [ ]:
train_list = [train_list_ml_small, train_list_ml_100k, train_list_ml_1m]
test_list = [test_list_ml_small, test_list_ml_100k, test_list_ml_1m]
cold_list = [cold_start_list_ml_small, cold_start_list_ml_100k, cold_start_list_ml_1m]

In [ ]:
print(np.count_nonzero(train_list_ml_small[0] > 0)/ train_list_ml_small[0].size)
print(np.count_nonzero(train_list_ml_100k[0] > 0)/ train_list_ml_100k[0].size)
print(np.count_nonzero(train_list_ml_1m[0] > 0)/ train_list_ml_1m[0].size)

0.0028332805092689374
0.010507992429353028
0.007447293274196488


# Experiment Setup

In [ ]:
dataset_list = ['01 ml_small','02 ml-100k','03 ml-1m']

k_user_list = [50]
k_item_list = [30]
min_support_list = [0.3]
min_confidence_list = [0.6]
rating_percentile_threshold_list = [0.8]

latent_list = [20, 50, 100]
learning_rate_list = [0.1, 0.01, 0.001]
lambda_rate_list = [0.1, 0.01, 0.001]
gamma_rate_list = [0.5, 0.1, 0.01]
epoch_list = [50, 100, 150]
batch_size_list = [512]

In [ ]:
signal_grid = list(itertools.product(
    k_user_list,
    k_item_list,
    min_support_list,
    min_confidence_list,
    rating_percentile_threshold_list
))

In [ ]:
MODEL_STANDARD_REGISTRY = {
    '01 MF': Standard_MF,
    '03 MF+Bias': Standard_MF_Bias
}

standard_model_grid = list(itertools.product(
    MODEL_STANDARD_REGISTRY.keys(),
    latent_list,
    learning_rate_list,
    lambda_rate_list,
    epoch_list,
    batch_size_list
))

In [ ]:
MODEL_CARMS_REGISTRY = {
    '02 CARMS_MF': CARMS_MF,
    '04 CARMS_MF+Bias': CARMS_MF_Bias
}

carms_model_grid = list(itertools.product(
    MODEL_CARMS_REGISTRY.keys(),
    latent_list,
    learning_rate_list,
    lambda_rate_list,
    gamma_rate_list,
    epoch_list,
    batch_size_list
))

# Model Training

In [ ]:
all_results = []

Base Model

In [ ]:
for i, dataset_name in enumerate(dataset_list):
    print(f"Data set: {dataset_name}")
    print('===========================================')
    train_list_data = train_list[i]
    test_list_data = test_list[i]
    cold_flag_list_data = cold_list[i]

    for fold in range(len(train_list_data)):
        print(f"Fold: {fold + 1}")
        train_array = train_list_data[fold]
        test_array = test_list_data[fold]
        cold_flag_array = cold_flag_list_data[fold]

        user_count = train_array.shape[0]
        item_count = train_array.shape[1]

        print(f"Shape of data: {train_array.shape}\n")

        # Model
        for model_params in standard_model_grid:
            model_name, latent, learning_rate, lambda_rate, epoch, batch_size = model_params
            print(f"--- Training Model: {model_name} ---")

            ModelClass = MODEL_STANDARD_REGISTRY[model_name]

            model_instance = ModelClass(user_count=user_count, item_count=item_count,
                                        K=latent, learning_rate=learning_rate, lambda_rate=lambda_rate)
            model_instance.fit(Y=train_array, epochs=epoch, batch_size=batch_size)
            results = model_instance.predict()
            pred_array = results['predictions']

            ndcg_5, ndcg_5_cold, ndcg_5_not_cold = \
            calculate_ndcg_at_k(pred_array, train_array, test_array, cold_flag_array, 5)
            ndcg_10, ndcg_10_cold, ndcg_10_not_cold = \
            calculate_ndcg_at_k(pred_array, train_array, test_array, cold_flag_array, 10)
            ndcg_20, ndcg_20_cold, ndcg_20_not_cold = \
            calculate_ndcg_at_k(pred_array, train_array, test_array, cold_flag_array, 20)

            # Save model
            result_entry = {
                'dataset_name': dataset_name,
                'fold': fold + 1,
                'model_name': model_name,

                'k_user': 99999,
                'k_item': 99999,
                'min_support': 99999,
                'min_confidence': 99999,
                'rating_percentile_threshold': 99999,

                'latent': latent,
                'learning_rate': learning_rate,
                'lambda_rate': lambda_rate,
                'gamma_rate':99999,
                'epoch': epoch,

                'ndcg_5': ndcg_5,
                'ndcg_5_cold': ndcg_5_cold,
                'ndcg_5_not_cold': ndcg_5_not_cold,

                'ndcg_10': ndcg_10,
                'ndcg_10_cold': ndcg_10_cold,
                'ndcg_10_not_cold': ndcg_10_not_cold,

                'ndcg_20': ndcg_20,
                'ndcg_20_cold': ndcg_20_cold,
                'ndcg_20_not_cold': ndcg_20_not_cold
                }

            all_results.append(result_entry)

Streaming output truncated to the last 5000 lines.
Epoch 55/150, Train RMSE (Y): 0.8435
Epoch 60/150, Train RMSE (Y): 0.8424
Epoch 65/150, Train RMSE (Y): 0.8341
Epoch 70/150, Train RMSE (Y): 0.8444
Epoch 75/150, Train RMSE (Y): 0.8278
Epoch 80/150, Train RMSE (Y): 0.8616
Epoch 85/150, Train RMSE (Y): 0.8290
Epoch 90/150, Train RMSE (Y): 0.8841
Epoch 95/150, Train RMSE (Y): 0.8568
Epoch 100/150, Train RMSE (Y): 0.8498
Epoch 105/150, Train RMSE (Y): 0.8460
Epoch 110/150, Train RMSE (Y): 0.8539
Epoch 115/150, Train RMSE (Y): 0.8584
Epoch 120/150, Train RMSE (Y): 0.8408
Epoch 125/150, Train RMSE (Y): 0.8255
Epoch 130/150, Train RMSE (Y): 0.8166
Epoch 135/150, Train RMSE (Y): 0.8676
Epoch 140/150, Train RMSE (Y): 0.8290
Epoch 145/150, Train RMSE (Y): 0.8464
Epoch 150/150, Train RMSE (Y): 0.8515
--- Training Model: 03 MF+Bias ---
Epoch 5/50, Train RMSE (Y): 0.9109
Epoch 10/50, Train RMSE (Y): 0.8519
Epoch 15/50, Train RMSE (Y): 0.8213
Epoch 20/50, Train RMSE (Y): 0.8291
Epoch 25/50, Train R

CARMS model

In [ ]:
for i, dataset_name in enumerate(dataset_list):
    print(f"Data set: {dataset_name}")
    print('===========================================')
    train_list_data = train_list[i]
    test_list_data = test_list[i]
    cold_flag_list_data = cold_list[i]

    for fold in range(len(train_list_data)):
        print(f"Fold: {fold + 1}")
        train_array = train_list_data[fold]
        test_array = test_list_data[fold]
        cold_flag_array = cold_flag_list_data[fold]

        user_count = train_array.shape[0]
        item_count = train_array.shape[1]

        print(f"Shape of data: {train_array.shape}\n")

        for signal_params in signal_grid:
            k_user, k_item, min_support, min_confidence, rating_percentile_threshold = signal_params
            signal_generator = SignalGenerator(k_user=k_user, k_item=k_item,
                                                min_support=min_support, min_confidence=min_confidence,
                                                rating_percentile_threshold=rating_percentile_threshold)
            S_matrix = signal_generator.fit_transform(train_array)

            # Model
            for model_params in carms_model_grid:
                model_name, latent, learning_rate, lambda_rate, gamma_rate, epoch, batch_size = model_params
                print(f"--- Training Model: {model_name} ---")

                ModelClass = MODEL_CARMS_REGISTRY[model_name]

                model_instance = ModelClass(user_count=user_count, item_count=item_count,
                                            K=latent, learning_rate=learning_rate, lambda_rate=lambda_rate,
                                            gamma=gamma_rate)
                model_instance.fit(Y=train_array, S=S_matrix, epochs=epoch, batch_size=batch_size)
                results = model_instance.predict()
                pred_array = results['predictions']

                ndcg_5, ndcg_5_cold, ndcg_5_not_cold = \
                calculate_ndcg_at_k(pred_array, train_array, test_array, cold_flag_array, 5)
                ndcg_10, ndcg_10_cold, ndcg_10_not_cold = \
                calculate_ndcg_at_k(pred_array, train_array, test_array, cold_flag_array, 10)
                ndcg_20, ndcg_20_cold, ndcg_20_not_cold = \
                calculate_ndcg_at_k(pred_array, train_array, test_array, cold_flag_array, 20)

                # Save model
                result_entry = {
                    'dataset_name': dataset_name,
                    'fold': fold + 1,
                    'model_name': model_name,

                    'k_user': k_user,
                    'k_item': k_item,
                    'min_support': min_support,
                    'min_confidence': min_confidence,
                    'rating_percentile_threshold': rating_percentile_threshold,

                    'latent': latent,
                    'learning_rate': learning_rate,
                    'lambda_rate': lambda_rate,
                    'gamma_rate':gamma_rate,
                    'epoch': epoch,

                    'ndcg_5': ndcg_5,
                    'ndcg_5_cold': ndcg_5_cold,
                    'ndcg_5_not_cold': ndcg_5_not_cold,

                    'ndcg_10': ndcg_10,
                    'ndcg_10_cold': ndcg_10_cold,
                    'ndcg_10_not_cold': ndcg_10_not_cold,

                    'ndcg_20': ndcg_20,
                    'ndcg_20_cold': ndcg_20_cold,
                    'ndcg_20_not_cold': ndcg_20_not_cold
                    }

                all_results.append(result_entry)

Streaming output truncated to the last 5000 lines.
Epoch 40/150, Train RMSE (Y): 0.9080, Train RMSE (S): 1.4523
Epoch 45/150, Train RMSE (Y): 0.9083, Train RMSE (S): 1.4468
Epoch 50/150, Train RMSE (Y): 0.9101, Train RMSE (S): 1.4434
Epoch 55/150, Train RMSE (Y): 0.9090, Train RMSE (S): 1.4381
Epoch 60/150, Train RMSE (Y): 0.9095, Train RMSE (S): 1.4408
Epoch 65/150, Train RMSE (Y): 0.9111, Train RMSE (S): 1.4433
Epoch 70/150, Train RMSE (Y): 0.9126, Train RMSE (S): 1.4484
Epoch 75/150, Train RMSE (Y): 0.9118, Train RMSE (S): 1.4510
Epoch 80/150, Train RMSE (Y): 0.9136, Train RMSE (S): 1.4752
Epoch 85/150, Train RMSE (Y): 0.9129, Train RMSE (S): 1.4587
Epoch 90/150, Train RMSE (Y): 0.9100, Train RMSE (S): 1.4692
Epoch 95/150, Train RMSE (Y): 0.9096, Train RMSE (S): 1.4729
Epoch 100/150, Train RMSE (Y): 0.9087, Train RMSE (S): 1.4715
Epoch 105/150, Train RMSE (Y): 0.9098, Train RMSE (S): 1.4458
Epoch 110/150, Train RMSE (Y): 0.9088, Train RMSE (S): 1.4347
Epoch 115/150, Train RMSE (Y): 

In [ ]:
results_df = pd.DataFrame(all_results)

# Summary Results

In [ ]:
results_final_df = duckdb.query("""
select dataset_name, model_name, latent, learning_rate, lambda_rate, gamma_rate, epoch
, k_user, k_item, min_support, min_confidence, rating_percentile_threshold

, ndcg_5, ndcg_5_cold, ndcg_5_not_cold
, ndcg_10, ndcg_10_cold, ndcg_10_not_cold
, ndcg_20, ndcg_20_cold, ndcg_20_not_cold
from
    (
    select *
    , row_number() over(partition by dataset_name, model_name order by ndcg_10 desc) rn
    from
        (
        select dataset_name, model_name, latent, learning_rate, lambda_rate, gamma_rate, epoch
        , k_user, k_item, min_support, min_confidence, rating_percentile_threshold
        , avg(ndcg_5) ndcg_5
        , avg(ndcg_5_cold) ndcg_5_cold
        , avg(ndcg_5_not_cold) ndcg_5_not_cold

        , avg(ndcg_10) ndcg_10
        , avg(ndcg_10_cold) ndcg_10_cold
        , avg(ndcg_10_not_cold) ndcg_10_not_cold

        , avg(ndcg_20) ndcg_20
        , avg(ndcg_20_cold) ndcg_20_cold
        , avg(ndcg_20_not_cold) ndcg_20_not_cold

        from results_df
        group by dataset_name, model_name, latent, learning_rate, lambda_rate, gamma_rate, epoch
        , k_user, k_item, min_support, min_confidence, rating_percentile_threshold
        ) a
    ) b
where rn = 1
order by dataset_name, model_name
""").to_df()

In [ ]:
results_final_df

,dataset_name,model_name,latent,learning_rate,lambda_rate,gamma_rate,epoch,k_user,k_item,min_support,...,rating_percentile_threshold,ndcg_5,ndcg_5_cold,ndcg_5_not_cold,ndcg_10,ndcg_10_cold,ndcg_10_not_cold,ndcg_20,ndcg_20_cold,ndcg_20_not_cold
0,01 ml_small,01 MF,100,0.001,0.100,99999.0,150,99999,99999,99999.0,...,99999.0,0.226989,0.232944,0.206871,0.208715,0.215233,0.184160,0.189174,0.194360,0.168441
1,01 ml_small,02 CARMS_MF,100,0.001,0.100,0.5,150,50,30,0.3,...,0.8,0.318871,0.352334,0.199673,0.289875,0.319120,0.182857,0.263423,0.288944,0.168498
2,01 ml_small,03 MF+Bias,20,0.001,0.001,99999.0,150,99999,99999,99999.0,...,99999.0,0.345778,0.380178,0.209407,0.317365,0.346308,0.195283,0.292925,0.321153,0.174950
3,01 ml_small,04 CARMS_MF+Bias,20,0.001,0.001,0.5,150,50,30,0.3,...,0.8,0.392165,0.437262,0.216394,0.372036,0.416282,0.200405,0.334977,0.375190,0.176890
4,02 ml-100k,01 MF,50,0.100,0.100,99999.0,50,99999,99999,99999.0,...,99999.0,0.225995,0.271488,0.138606,0.209051,0.246518,0.137462,0.189709,0.216731,0.136085
5,02 ml-100k,02 CARMS_MF,100,0.001,0.100,0.5,100,50,30,0.3,...,0.8,0.275732,0.355249,0.132751,0.265928,0.342845,0.129171,0.248616,0.315177,0.130033
6,02 ml-100k,03 MF+Bias,20,0.001,0.010,99999.0,50,99999,99999,99999.0,...,99999.0,0.317371,0.418070,0.131913,0.303629,0.398305,0.128417,0.283086,0.365583,0.128066
7,02 ml-100k,04 CARMS_MF+Bias,20,0.001,0.010,0.5,50,50,30,0.3,...,0.8,0.340596,0.449839,0.136192,0.323135,0.424799,0.131104,0.297020,0.386775,0.129448
8,03 ml-1m,01 MF,50,0.001,0.100,99999.0,100,99999,99999,99999.0,...,99999.0,0.223543,0.228699,0.183657,0.212446,0.218299,0.172932,0.200634,0.204843,0.166309
9,03 ml-1m,02 CARMS_MF,20,0.001,0.100,0.1,50,50,30,0.3,...,0.8,0.303252,0.362666,0.184511,0.289656,0.347962,0.173483,0.273680,0.327064,0.166490


In [ ]:
results_df.to_excel('./results/df_result_carms.xlsx', index=False)
results_final_df.to_excel('./results/df_results_best_carms.xlsx', index=False)